In [36]:
# Connect to google drive to access datasets
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
import numpy as np
import cv2
import os
from google.colab.patches import cv2_imshow

In [38]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,Dense, MaxPool2D
from keras.activations import relu,sigmoid
from keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split,KFold

In [39]:
# Let's Import First Dataset: MaskedFaceNet
FirstCorrectData=np.load("/content/drive/MyDrive/MSProjectData/Dataset1/Correct/correct_features.npz")
FirstIncorrectData=np.load("/content/drive/MyDrive/MSProjectData/Dataset1/Incorrect/incorrect_features.npz")
# Extract Correctly Masked Features from npz 
FirstCorrectData=FirstCorrectData['arr_0']
FirstCorrectData=FirstCorrectData/255
# Extract Incorrectly Masked Features from npz
FirstIncorrectData=FirstIncorrectData['arr_0']
FirstIncorrectData=FirstIncorrectData/255
print(FirstCorrectData.shape,FirstIncorrectData.shape)

(67193, 16, 16, 1) (67857, 16, 16, 1)


In [40]:
# Let's Import Second Dataset: Kaggle
CorrectFeatures=np.load("/content/drive/MyDrive/MSProjectData/Dataset2/Correct/correct_features.npz")
IncorrectFeatures=np.load("/content/drive/MyDrive/MSProjectData/Dataset2/Incorrect/incorrect_features.npz")
# Extract Correctly Masked Features from npz 
CorrectFeatures=CorrectFeatures['arr_0']
CorrectFeatures=CorrectFeatures/255
# Extract Incorrectly Masked Features from npz
IncorrectFeatures=IncorrectFeatures['arr_0']
IncorrectFeatures=IncorrectFeatures/255
print(CorrectFeatures.shape,IncorrectFeatures.shape)

(2287, 16, 16, 1) (550, 16, 16, 1)


In [41]:
# Let's import third Dataset: SCFace
# Load Correctly Masked Faces
correct_data=np.load("/content/drive/MyDrive/MSProjectData/Dataset3/Correct/correct_features.npz")
# Load Incorrectly Masked Faces
incorrect_data=np.load("/content/drive/MyDrive/MSProjectData/Dataset3/Incorrect/incorrect_features.npz")
# Extract Correctly Masked Features from npz 
correct_data=correct_data['arr_0']
correct_data=correct_data/255
correct_data=np.reshape(correct_data,(correct_data.shape[0]*correct_data.shape[1],correct_data.shape[2],correct_data.shape[3],correct_data.shape[4]))
# Extract Incorrectly Masked Features from npz
incorrect_data=incorrect_data['arr_0']
incorrect_data=incorrect_data/255
incorrect_data=np.reshape(incorrect_data,(incorrect_data.shape[0]*incorrect_data.shape[1],incorrect_data.shape[2],incorrect_data.shape[3],incorrect_data.shape[4]))
print(correct_data.shape,incorrect_data.shape)

(2844, 16, 16, 1) (3081, 16, 16, 1)


In [42]:
# Let's merge all the data
CompleteCorrectFeatures=np.concatenate((correct_data,CorrectFeatures,FirstCorrectData),axis=0)
CompleteIncorrectFeatures=np.concatenate((incorrect_data,IncorrectFeatures,FirstIncorrectData),axis=0)
print(CompleteCorrectFeatures.shape,CompleteIncorrectFeatures.shape)

(72324, 16, 16, 1) (71488, 16, 16, 1)


In [43]:
# Let's merge all labels
CompleteCorrectLabels=np.zeros(CompleteCorrectFeatures.shape[0])
CompleteIncorrectLabels=np.ones(CompleteIncorrectFeatures.shape[0])
print(CompleteCorrectLabels.shape,CompleteIncorrectLabels.shape)

(72324,) (71488,)


In [44]:
CompleteFeatures=np.concatenate((CompleteCorrectFeatures,CompleteIncorrectFeatures),axis=0)
CompleteFeatures=np.reshape(CompleteFeatures,(CompleteFeatures.shape[0],CompleteFeatures.shape[1],CompleteFeatures.shape[2],1))
CompleteLabels=np.concatenate((CompleteCorrectLabels,CompleteIncorrectLabels),axis=0)
print(CompleteFeatures.shape,CompleteLabels.shape)

(143812, 16, 16, 1) (143812,)


In [45]:
kFold=KFold(n_splits=5,shuffle=True)

CorrectDataArrTrain=[]
CorrectDataArrTest=[]
for train,test in kFold.split(correct_data):
  CorrectDataArrTrain.append(correct_data[train])
  CorrectDataArrTest.append(correct_data[test])
CorrectDataArrTrain=np.asarray(CorrectDataArrTrain)
CorrectDataArrTest=np.asarray(CorrectDataArrTest)

IncorrectDataArrTrain=[]
IncorrectDataArrTest=[]
for train,test in kFold.split(incorrect_data):
  IncorrectDataArrTrain.append(incorrect_data[train])
  IncorrectDataArrTest.append(incorrect_data[test])
IncorrectDataArrTrain=np.asarray(IncorrectDataArrTrain)
IncorrectDataArrTest=np.asarray(IncorrectDataArrTest)

CorrectFeaturesArrTrain=[]
CorrectFeaturesArrTest=[]
for train,test in kFold.split(CorrectFeatures):
  CorrectFeaturesArrTrain.append(CorrectFeatures[train])
  CorrectFeaturesArrTest.append(CorrectFeatures[test])
CorrectFeaturesArrTrain=np.asarray(CorrectFeaturesArrTrain)
CorrectFeaturesArrTest=np.asarray(CorrectFeaturesArrTest)

IncorrectFeaturesArrTrain=[]
IncorrectFeaturesArrTest=[]
for train,test in kFold.split(IncorrectFeatures):
  IncorrectFeaturesArrTrain.append(IncorrectFeatures[train])
  IncorrectFeaturesArrTest.append(IncorrectFeatures[test])
IncorrectFeaturesArrTrain=np.asarray(IncorrectFeaturesArrTrain)
IncorrectFeaturesArrTest=np.asarray(IncorrectFeaturesArrTest)

FirstCorrectFeaturesArrTrain=[]
FirstCorrectFeaturesArrTest=[]
for train,test in kFold.split(FirstCorrectData):
  FirstCorrectFeaturesArrTrain.append(FirstCorrectData[train])
  FirstCorrectFeaturesArrTest.append(FirstCorrectData[test])
FirstCorrectFeaturesArrTrain=np.asarray(FirstCorrectFeaturesArrTrain)
FirstCorrectFeaturesArrTest=np.asarray(FirstCorrectFeaturesArrTest)

FirstIncorrectFeaturesArrTrain=[]
FirstIncorrectFeaturesArrTest=[]
for train,test in kFold.split(FirstIncorrectData):
  FirstIncorrectFeaturesArrTrain.append(FirstIncorrectData[train])
  FirstIncorrectFeaturesArrTest.append(FirstIncorrectData[test])
FirstIncorrectFeaturesArrTrain=np.asarray(FirstIncorrectFeaturesArrTrain)
FirstIncorrectFeaturesArrTest=np.asarray(FirstIncorrectFeaturesArrTest)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [46]:
print(CorrectDataArrTrain[0].shape,IncorrectDataArrTrain[0].shape,CorrectFeaturesArrTrain[0].shape,IncorrectFeaturesArrTrain[0].shape,FirstCorrectFeaturesArrTrain[0].shape,FirstIncorrectFeaturesArrTrain[0].shape)

(2275, 16, 16, 1) (2464, 16, 16, 1) (1829, 16, 16, 1) (440, 16, 16, 1) (53754, 16, 16, 1) (54285, 16, 16, 1)


In [47]:
totalAccuracy=[]
totalCorrectDataset1=[]
totalIncorrectDataset1=[]
totalCorrectDataset2=[]
totalIncorrectDataset2=[]
totalCorrectDataset3=[]
totalIncorrectDataset3=[]
for i in range(0,5):
  print("###################",i,"###################")
  x_train=np.concatenate((CorrectDataArrTrain[i],IncorrectDataArrTrain[i],CorrectFeaturesArrTrain[i],IncorrectFeaturesArrTrain[i],FirstCorrectFeaturesArrTrain[i],FirstIncorrectFeaturesArrTrain[i]),axis=0)
  x_test=np.concatenate((CorrectDataArrTest[i],IncorrectDataArrTest[i],CorrectFeaturesArrTest[i],IncorrectFeaturesArrTest[i],FirstCorrectFeaturesArrTest[i],FirstIncorrectFeaturesArrTest[i]),axis=0)
  y_train=np.concatenate((np.zeros(CorrectDataArrTrain[i].shape[0]),np.ones(IncorrectDataArrTrain[i].shape[0]),np.zeros(CorrectFeaturesArrTrain[i].shape[0]),np.ones(IncorrectFeaturesArrTrain[i].shape[0]),
                        np.zeros(FirstCorrectFeaturesArrTrain[i].shape[0]),np.ones(FirstIncorrectFeaturesArrTrain[i].shape[0])),axis=0)
  y_test=np.concatenate((np.zeros(CorrectDataArrTest[i].shape[0]),np.ones(IncorrectDataArrTest[i].shape[0]),np.zeros(CorrectFeaturesArrTest[i].shape[0]),np.ones(IncorrectFeaturesArrTest[i].shape[0]),
                        np.zeros(FirstCorrectFeaturesArrTest[i].shape[0]),np.ones(FirstIncorrectFeaturesArrTest[i].shape[0])),axis=0)
  x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],x_train.shape[2],1))
  x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],x_test.shape[2],1))
  print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)
  model=Sequential()
  model.add(Conv2D(filters=128,kernel_size=(5,5),activation='relu',input_shape=x_train[0].shape))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu'))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Flatten())
  model.add(Dense(units=128,activation='relu'))
  model.add(Dense(units=32,activation='relu'))
  model.add(Dense(units=1,activation='sigmoid'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  class MyClass(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if logs.get('accuracy') > 0.995 :
        self.model.stop_training=True
  model.fit(x_train,y_train,epochs=500,batch_size=128,shuffle=True,callbacks=[MyClass()])
  totalAccuracy.append(model.evaluate(x_test,y_test)[1])
  totalCorrectDataset1.append(model.evaluate(np.reshape(CorrectDataArrTest[i],(CorrectDataArrTest[i].shape[0],CorrectDataArrTest[i].shape[1],CorrectDataArrTest[i].shape[2],1)),np.zeros(CorrectDataArrTest[i].shape[0]))[1])
  totalIncorrectDataset1.append(model.evaluate(np.reshape(IncorrectDataArrTest[i],(IncorrectDataArrTest[i].shape[0],IncorrectDataArrTest[i].shape[1],IncorrectDataArrTest[i].shape[2],1)),np.ones(IncorrectDataArrTest[i].shape[0]))[1])
  totalCorrectDataset2.append(model.evaluate(np.reshape(CorrectFeaturesArrTest[i],(CorrectFeaturesArrTest[i].shape[0],CorrectFeaturesArrTest[i].shape[1],CorrectFeaturesArrTest[i].shape[2],1)),np.zeros(CorrectFeaturesArrTest[i].shape[0]))[1])
  totalIncorrectDataset2.append(model.evaluate(np.reshape(IncorrectFeaturesArrTest[i],(IncorrectFeaturesArrTest[i].shape[0],IncorrectFeaturesArrTest[i].shape[1],IncorrectFeaturesArrTest[i].shape[2],1)),np.ones(IncorrectFeaturesArrTest[i].shape[0]))[1])
  totalCorrectDataset3.append(model.evaluate(np.reshape(FirstCorrectFeaturesArrTest[i],(FirstCorrectFeaturesArrTest[i].shape[0],FirstCorrectFeaturesArrTest[i].shape[1],FirstCorrectFeaturesArrTest[i].shape[2],1)),np.zeros(FirstCorrectFeaturesArrTest[i].shape[0]))[1])
  totalIncorrectDataset3.append(model.evaluate(np.reshape(FirstIncorrectFeaturesArrTest[i],(FirstIncorrectFeaturesArrTest[i].shape[0],FirstIncorrectFeaturesArrTest[i].shape[1],FirstIncorrectFeaturesArrTest[i].shape[2],1)),np.ones(FirstIncorrectFeaturesArrTest[i].shape[0]))[1])

################### 0 ###################
(115047, 16, 16, 1) (115047,) (28765, 16, 16, 1) (28765,)
Epoch 1/500
899/899 [==============================] - 3s 3ms/step - loss: 0.3008 - accuracy: 0.8603
Epoch 2/500
899/899 [==============================] - 3s 3ms/step - loss: 0.1278 - accuracy: 0.9518
Epoch 3/500
899/899 [==============================] - 3s 3ms/step - loss: 0.1025 - accuracy: 0.9605
Epoch 4/500
899/899 [==============================] - 3s 3ms/step - loss: 0.0869 - accuracy: 0.9672
Epoch 5/500
899/899 [==============================] - 3s 3ms/step - loss: 0.0815 - accuracy: 0.9695
Epoch 6/500
899/899 [==============================] - 3s 3ms/step - loss: 0.0744 - accuracy: 0.9716
Epoch 7/500
899/899 [==============================] - 3s 3ms/step - loss: 0.0671 - accuracy: 0.9758
Epoch 8/500
899/899 [==============================] - 3s 3ms/step - loss: 0.0655 - accuracy: 0.9759
Epoch 9/500
899/899 [==============================] - 3s 3ms/step - loss: 0.0605 - accuracy

In [48]:
print("totalAccuracy",(sum(totalAccuracy)/len(totalAccuracy))*100)
print("totalCorrectDataset1",(sum(totalCorrectDataset1)/len(totalCorrectDataset1))*100)
print("totalIncorrectDataset1",(sum(totalIncorrectDataset1)/len(totalIncorrectDataset1))*100)
print("totalCorrectDataset2",(sum(totalCorrectDataset2)/len(totalCorrectDataset2))*100)
print("totalIncorrectDataset2",(sum(totalIncorrectDataset2)/len(totalIncorrectDataset2))*100)
print("totalCorrectDataset3",(sum(totalCorrectDataset3)/len(totalCorrectDataset3))*100)
print("totalIncorrectDataset3",(sum(totalIncorrectDataset3)/len(totalIncorrectDataset3))*100)

totalAccuracy 97.57808327674866
totalCorrectDataset1 99.29682850837708
totalIncorrectDataset1 98.08499336242676
totalCorrectDataset2 87.93144941329956
totalIncorrectDataset2 79.81818318367004
totalCorrectDataset3 97.64708042144775
totalIncorrectDataset3 97.88379073143005


In [51]:
model.save("/content/drive/MyDrive/MSProjectData/Experiment4Merge3Datasets.h5")